In [1]:
import pandas as pd
import numpy as np
import torch

import warnings
#warnings.filterwarnings('ignore')
#warnings.simplefilter('ignore')


def read_file(fname: str) -> pd.DataFrame:
    """Reads a filename and formats it properly for simpletransformers"""
    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    offensive_ids = df.labels != "Acceptable speech"

    df.labels[offensive_ids] = 1
    df.labels[~offensive_ids] = 0
    
    df["labels"] = df.labels.astype(np.int8)
    df = df.drop(columns=["role"])
    return df

def fine_tune_and_evaluate(
    model_type,
    model_name,
    language,
    model_args
                            ):
    import torch
    torch.cuda.empty_cache()
    if language not in {"sl", "hr", "en"}:
        raise AttributeError(f"Language {language} is not valid")
    eval_file, train_file = f"../data/merged-{language}.test.tsv" , f"../data/merged-{language}.train.tsv"
    train = read_file(train_file)
    test = read_file(eval_file)
    
    from simpletransformers.classification import ClassificationModel
    model_args["overwrite_output_dir"] = True
    model_args["output_dir"] = f"finetuned_models/{model_type}__{model_name}_{language}_current/"
    model_args["silent"] = False
    model_args["save_model_every_epoch"] = False
    model_args["save_steps"] = -1
    #model_args["manual_seed"] = 42
    model = ClassificationModel(
        model_type, model_name, use_cuda=True,
        args=model_args

    )
    
    def copy_best_model(model_type, model_name, language):
        import os
        os.system(f"cp -r finetuned_models/{model_type}__{model_name}_{language}_current finetuned_models/{model_type}__{model_name}_{language}_best")
    accuracies = list()
    f1s = list()
    
    
    model.train_model(train, )
    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["labels"]
    y_pred = model.predict(list(test["text"].values))[0]
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro")
    
    accuracies.append(accuracy)
    f1s.append(f1)
    
    for i in range(6):
        model = ClassificationModel(model_type, model_name, use_cuda=True, args=model_args)
        model.train_model(train, )
        y_true = test["labels"]
        y_pred = model.predict(list(test["text"].values))[0]
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average="macro")
    
        accuracies.append(accuracy)
        f1s.append(f1)
        
        if accuracy == max(accuracies):
            copy_best_model(model_type, model_name, language)
    print(f"Model: {model_name}, {model_type}, {language=}")
    print(f"Accuracies: {accuracies}")
    print(f"F1 scores: {f1s}")
    return #model, accuracy, f1


roberta_args = {
        "num_train_epochs": 6,
        "learning_rate": 3e-6,
        "train_batch_size": 69}
embeddia_args = {
        "num_train_epochs": 14,
        "learning_rate": 1e-5,
        "train_batch_size": 21}
bertic_args = {
        "num_train_epochs": 12,
        "learning_rate": 1e-5,
        "train_batch_size": 74}


config_dict = {
    "sl": [
        ("xlm-roberta-base", "xlmroberta", roberta_args),
        ("EMBEDDIA/sloberta", "camembert", embeddia_args),
        #("EMBEDDIA/sloberta", "roberta", embeddia_args),
        ("EMBEDDIA/crosloengual-bert", "bert", embeddia_args),
        ],
    "hr": [
        ("xlm-roberta-base", "xlmroberta", roberta_args),
        ("classla/bcms-bertic", "electra", bertic_args),
        ("EMBEDDIA/crosloengual-bert", "bert", embeddia_args),
    ],
    "en": [
        ("xlm-roberta-base", "xlmroberta", roberta_args),
        ("xlm-roberta-large", "xlmroberta", roberta_args),
        ("roberta-base", "roberta", roberta_args),
    ]
}

for language, model_list in config_dict.items():
    for model_name, model_type, model_args in model_list:
        fine_tune_and_evaluate(model_type,
                                model_name,
                                language,
                                model_args)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.w

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.w

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.w

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.w

...

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.


Model: roberta-base, roberta, language='en'
Accuracies: [0.7927888792354474, 0.7962641181581234, 0.790616854908775, 0.7819287576020851, 0.7953953084274544, 0.792354474370113, 0.7910512597741095]
F1 scores: [0.779903005739679, 0.7842603092706253, 0.7780383684410571, 0.7678973127920354, 0.7823783904211664, 0.7787919676803026, 0.7783526833497377]


# Continuation: added another english model

In [3]:
from simpletransformers.classification import ClassificationModel

model_args = {
    "num_train_epochs": 5,
    "learning_rate": 1e-5,
    "overwrite_output_dir": True,
    "train_batch_size": 40
}

model = ClassificationModel(
    "distilbert", "distilbert-base-uncased-finetuned-sst-2-english", use_cuda=True,
    args=model_args
    
)


In [4]:
import pandas as pd
import numpy as np
import torch

import warnings
#warnings.filterwarnings('ignore')
#warnings.simplefilter('ignore')


def read_file(fname: str) -> pd.DataFrame:
    """Reads a filename and formats it properly for simpletransformers"""
    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    offensive_ids = df.labels != "Acceptable speech"

    df.labels[offensive_ids] = 1
    df.labels[~offensive_ids] = 0
    
    df["labels"] = df.labels.astype(np.int8)
    df = df.drop(columns=["role"])
    return df

def fine_tune_and_evaluate(
    model_type,
    model_name,
    language,
    model_args
                            ):
    import torch
    torch.cuda.empty_cache()
    if language not in {"sl", "hr", "en"}:
        raise AttributeError(f"Language {language} is not valid")
    eval_file, train_file = f"../data/merged-{language}.test.tsv" , f"../data/merged-{language}.train.tsv"
    train = read_file(train_file)
    test = read_file(eval_file)
    
    from simpletransformers.classification import ClassificationModel
    model_args["overwrite_output_dir"] = True
    model_args["output_dir"] = f"finetuned_models/{model_type}__{model_name}_{language}_current/"
    model_args["silent"] = False
    model_args["save_model_every_epoch"] = False
    model_args["save_steps"] = -1
    #model_args["manual_seed"] = 42
    model = ClassificationModel(
        model_type, model_name, use_cuda=True,
        args=model_args

    )
    
    def copy_best_model(model_type, model_name, language):
        import os
        os.system(f"cp -r finetuned_models/{model_type}__{model_name}_{language}_current finetuned_models/{model_type}__{model_name}_{language}_best")
    accuracies = list()
    f1s = list()
    
    
    model.train_model(train, )
    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["labels"]
    y_pred = model.predict(list(test["text"].values))[0]
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro")
    
    accuracies.append(accuracy)
    f1s.append(f1)
    
    for i in range(6):
        model = ClassificationModel(model_type, model_name, use_cuda=True, args=model_args)
        model.train_model(train, )
        y_true = test["labels"]
        y_pred = model.predict(list(test["text"].values))[0]
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average="macro")
    
        accuracies.append(accuracy)
        f1s.append(f1)
        
        if accuracy == max(accuracies):
            copy_best_model(model_type, model_name, language)
    print(f"Model: {model_name}, {model_type}, {language=}")
    print(f"Accuracies: {accuracies}")
    print(f"F1 scores: {f1s}")
    return #model, accuracy, f1


roberta_args = {
        "num_train_epochs": 6,
        "learning_rate": 3e-6,
        "train_batch_size": 69}
embeddia_args = {
        "num_train_epochs": 14,
        "learning_rate": 1e-5,
        "train_batch_size": 21}
bertic_args = {
        "num_train_epochs": 12,
        "learning_rate": 1e-5,
        "train_batch_size": 74}


config_dict = {
    #"sl": [
    #    ("xlm-roberta-base", "xlmroberta", roberta_args),
    #    ("EMBEDDIA/sloberta", "camembert", embeddia_args),
        #("EMBEDDIA/sloberta", "roberta", embeddia_args),
    #    ("EMBEDDIA/crosloengual-bert", "bert", embeddia_args),
    #    ],
    #"hr": [
    #    ("xlm-roberta-base", "xlmroberta", roberta_args),
    #    ("classla/bcms-bertic", "electra", bertic_args),
    #    ("EMBEDDIA/crosloengual-bert", "bert", embeddia_args),
    #],
    "en": [
    #    ("xlm-roberta-base", "xlmroberta", roberta_args),
    #    ("xlm-roberta-large", "xlmroberta", roberta_args),
    #    ("roberta-base", "roberta", roberta_args),
        ("distilbert-base-uncased-finetuned-sst-2-english", "distilbert", bertic_args)
    ]
}

for language, model_list in config_dict.items():
    for model_name, model_type, model_args in model_list:
        fine_tune_and_evaluate(model_type,
                                model_name,
                                language,
                                model_args)


Model: distilbert-base-uncased-finetuned-sst-2-english, distilbert, language='en'
Accuracies: [0.7232841007819287, 0.7132927888792354, 0.7224152910512598, 0.7158992180712423, 0.7254561251086012, 0.7193744569939183, 0.7211120764552563]
F1 scores: [0.7044579789396375, 0.6905553514298262, 0.6993976623782681, 0.6936789340938113, 0.7055884733689493, 0.6957070128339444, 0.7009300631374454]
